In [3]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector
from keras.layers.normalization.batch_normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
%matplotlib inline

ModuleNotFoundError: No module named 'tensorflow'

In [15]:
import pandas as pd
import numpy as np
from glob import glob

In [68]:
def readFile(file):
  train = pd.read_csv(file)
  return train

def buildTrainXY(train_x, train_y):
  X_train, Y_train = [], []
  X_train = [train_x.iloc[0]['pos_x'], train_x.iloc[0]['pos_y'], train_x.iloc[0]['vel_x'], train_x.iloc[0]['vel_y']]
  y_lst = []
  for i in range(len(train_y)):
    # y_lst.append([train_y.iloc[i]['pos_x'], train_y.iloc[i]['pos_y'], train_y.iloc[i]['vel_x'], train_y.iloc[i]['vel_y']])
    y_lst.append(train_y.iloc[i]['pos_x'])
    y_lst.append(train_y.iloc[i]['pos_y'])
    y_lst.append(train_y.iloc[i]['vel_x'])
    y_lst.append(train_y.iloc[i]['vel_y'])
  Y_train.append(y_lst)
  
  Y_train = list(np.array(Y_train).flat)
  return X_train, Y_train

def buildTrainAG(train_ag):
  AG_train = train_ag.iloc[0]
  return AG_train

In [70]:
folder = "/home/linsea/motion_ws/src/motion_prediction/data/ETH_v1/"
X_train ,Y_train , AG_train= [], [], []

for file in glob(os.path.join(folder+'train_x/',"*.csv")):
  file_name = file.split('/')[-1]
  train_x = readFile(file)
  train_y = readFile(folder + 'train_y/' + file_name)
  train_ag = readFile(folder + 'train_ag/' + file_name)
  X, Y = buildTrainXY(train_x, train_y)
  AG = buildTrainAG(train_ag)
  X_train.append(X)
  Y_train.append(Y)
  AG_train.append(AG)



[2.678267, 2.521993, 1.5636247, 0.50020444, 3.3765585, 2.7875629, 1.6036229, 0.54846313, 3.9611653, 2.9607635, 1.4429478, 0.32021265, 4.5309168, 3.043733, 1.416975, 0.26011473, 5.0947453, 3.1688553, 1.4553399, 0.30969665]
[2.678267, 2.521993, 1.5636247, 0.50020444, 3.3765585, 2.7875629, 1.6036229, 0.54846313, 3.9611653, 2.9607635, 1.4429478, 0.32021265, 4.5309168, 3.043733, 1.416975, 0.26011473, 5.0947453, 3.1688553, 1.4553399, 0.30969665]
[0.43496351, 6.1000501, 1.9315392, -0.092024557, 1.0532273, 6.1612846, 1.5873768, 0.037735519, 1.704865, 6.1302385, 1.6811024, -0.024806482, 2.3981092, 6.1414394, 1.781737, 0.079393791, 3.1302545, 6.1937535, 1.7469798, 0.024366582]
[0.43496351, 6.1000501, 1.9315392, -0.092024557, 1.0532273, 6.1612846, 1.5873768, 0.037735519, 1.704865, 6.1302385, 1.6811024, -0.024806482, 2.3981092, 6.1414394, 1.781737, 0.079393791, 3.1302545, 6.1937535, 1.7469798, 0.024366582]
[-4.5589337, -0.031150824, 1.1316956, 1.0890524, -4.1115353, 0.37675707, 1.1115865, 1.013470

In [72]:
X_train = pd.DataFrame(X_train)
Y_train = pd.DataFrame(Y_train)
AG_train = pd.DataFrame(AG_train)
Y_train

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,2.678267,2.521993,1.563625,0.500204,3.376558,2.787563,1.603623,0.548463,3.961165,2.960764,...,1.416975,0.260115,5.094745,3.168855,1.455340,0.309697,NaN,NaN,NaN,NaN
1,0.434964,6.100050,1.931539,-0.092025,1.053227,6.161285,1.587377,0.037736,1.704865,6.130238,...,1.781737,0.079394,3.130254,6.193753,1.746980,0.024367,NaN,NaN,NaN,NaN
2,-4.558934,-0.031151,1.131696,1.089052,-4.111535,0.376757,1.111587,1.013470,-3.669665,0.779625,...,1.488159,0.811711,-2.479137,1.428994,1.547262,0.978593,NaN,NaN,NaN,NaN
3,8.093101,8.835435,0.000000,0.000000,8.093101,8.835435,0.000000,0.000000,8.093101,8.835435,...,0.000000,0.000000,8.093101,8.835435,0.000000,0.000000,NaN,NaN,NaN,NaN
4,8.093101,8.835435,0.000000,0.000000,8.093101,8.835435,0.000000,0.000000,8.093101,8.835435,...,0.000000,0.000000,8.093101,8.835435,0.000000,0.000000,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1424,5.080252,5.155564,1.614685,0.208997,5.764804,5.215332,1.585437,0.152382,6.348602,5.277469,...,1.523870,-0.007916,7.567698,5.271136,1.605478,-0.013397,NaN,NaN,NaN,NaN
1425,11.417994,6.136983,1.326452,-0.462365,11.905002,5.916416,1.304614,-0.504345,12.461685,5.733507,...,1.380306,-0.163747,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1426,3.037348,4.169850,-1.484567,-0.463714,2.440227,4.004992,-1.442724,-0.416699,1.883168,3.836491,...,-1.553152,-0.429112,0.640647,3.493201,-1.448912,-0.438820,NaN,NaN,NaN,NaN
1427,8.625631,6.166830,1.540472,-0.335661,9.237792,6.053935,1.862856,-0.397031,10.115915,5.849205,...,1.558858,-0.174875,11.363002,5.709305,1.482858,-0.218944,NaN,NaN,NaN,NaN


In [74]:
AG_train

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
0,1.0000,1.0,1.0,1.0000,1.0,1.0,1.0,1.0,1.0,1.0000,...,1.0000,1.0,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
0,1.0000,1.0,1.0,0.4319,1.0,1.0,1.0,1.0,1.0,1.0000,...,0.2313,1.0,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.5782,1.0000
0,1.0000,1.0,1.0,1.0000,1.0,1.0,1.0,1.0,1.0,1.0000,...,1.0000,1.0,0.1108,0.2491,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
0,1.0000,1.0,1.0,1.0000,1.0,1.0,1.0,1.0,1.0,1.0000,...,0.3617,1.0,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
0,1.0000,1.0,1.0,1.0000,1.0,1.0,1.0,1.0,1.0,1.0000,...,1.0000,1.0,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.5155,1.0,1.0,1.0000,1.0,1.0,1.0,1.0,1.0,1.0000,...,1.0000,1.0,1.0000,0.2629,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
0,1.0000,1.0,1.0,1.0000,1.0,1.0,1.0,1.0,1.0,1.0000,...,1.0000,1.0,1.0000,1.0000,1.0000,1.0000,0.3382,1.0000,1.0000,1.0000
0,1.0000,1.0,1.0,0.6129,1.0,1.0,1.0,1.0,1.0,0.7817,...,1.0000,1.0,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
0,1.0000,1.0,1.0,1.0000,1.0,1.0,1.0,1.0,1.0,1.0000,...,1.0000,1.0,1.0000,1.0000,0.3240,1.0000,1.0000,0.2737,1.0000,0.5778


In [17]:
# def buildTrain(train):
X_train, Y_train = [], []
x_lst = [train_x.iloc[0]['pos_x'], train_x.iloc[0]['pos_y'], train_x.iloc[0]['vel_x'], train_x.iloc[0]['vel_y']]
X_train.append(np.array(x_lst))
y_lst = []
for i in range(len(train_y)):
  y_lst.append(train_y.iloc[i]['pos_x'])
  y_lst.append(train_y.iloc[i]['pos_y'])
  y_lst.append(train_y.iloc[i]['vel_x'])
  y_lst.append(train_y.iloc[i]['vel_y'])
Y_train.append(np.array(y_lst))
  # return np.array(X_train), np.array(Y_train)

[array([ 9.1255301 ,  3.6585832 ,  1.6628772 ,  0.32672255,  9.787146  ,
        3.8494445 ,  1.6833339 ,  0.37108399, 10.472197  ,  3.9554504 ,
        1.5985677 ,  0.26479471, 11.066     ,  4.0612803 ,  1.5745265 ,
        0.45639045, 11.731818  ,  4.3205627 ,  1.644127  ,  0.54439117])]
